In [1]:
import sys
from pathlib import Path

# add parent folder (logic_circuits) to sys.path
sys.path.append(str(Path.cwd().parent))


In [2]:
import pygame
from pygame.math import Vector2

import logic_circuits.pygame_representation.fonts as fonts
from logic_circuits.pygame_representation.colors import BG, TEXT, WIRE_HOT
from logic_circuits.pygame_representation.utils import draw_grid, cubic_bezier
from logic_circuits.pygame_representation.gates_graphical import GateAND_graphical, GateNOT_graphical, GatePass_graphical, SysIN_graphical
from logic_circuits.pygame_representation.wires import Wire

# initialize
pygame.init()
fonts.init()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
from logic_circuits.utils.base_classes import GateBase
from logic_circuits.gates.gates import SysIN, GateNOT, GateAND, GatePASS, truthtable, _read, make_combined_gate_class

from typing import Protocol, Iterable, List, Tuple, Union
import numpy as np

In [4]:
sysin = SysIN_graphical(name = "SysIN", num_in=2, num_out=2, x = 100, y = 200)
sysout = GatePass_graphical(name = "SysOUT", num_in=2, num_out=2, x=800, y = 200)
connections = []

Block0 = GateNOT_graphical("notgate1", x = 150, y = 200, w = 100, h = 100)
Block1 = GateNOT_graphical("notgate2", x = 300, y = 200, w = 100, h = 100)
Block2 = GateNOT_graphical("notgate3", x = 300, y = 200, w = 100, h = 100)

Block3 = GateAND_graphical("andgate1", x = 150, y = 200, w = 100, h = 100)
# Block4 = GateAND_graphical("andgate2", x = 300, y = 200, w = 100, h = 100)
# Block5 = GateAND_graphical("andgate3", x = 300, y = 200, w = 100, h = 100)

blocks = [sysin, Block0, Block1, Block2, Block3,  sysout]

# Interaction state
wires = []
drag_start_block = None
drag_start_block = None
drag_start_port_g = None
drag_stop_port_g = None

drag_start_gate_g = None
drag_stop_gate_g = None

drag_start_port_g = Vector2(0, 0)
port_drag_pos = Vector2(0, 0)

import pygame
from pygame.math import Vector2

from logic_circuits.pygame_representation import fonts
from logic_circuits.pygame_representation.colors import BG, TEXT, WIRE_HOT
from logic_circuits.pygame_representation.utils import draw_grid, cubic_bezier
from logic_circuits.pygame_representation.wires import Wire


In [5]:
sysin = SysIN_graphical(name = "SysIN", num_in=2, num_out=2, x = 100, y = 200)
sysout = GatePass_graphical(name = "SysOUT", num_in=2, num_out=2, x=800, y = 200)
connections = []

# Block0 = GateNOT_graphical("notgate1", x = 150, y = 200, w = 100, h = 100)
# Block1 = GateNOT_graphical("notgate2", x = 300, y = 200, w = 100, h = 100)
# Block2 = GateNOT_graphical("notgate3", x = 300, y = 200, w = 100, h = 100)

# Block3 = GateAND_graphical("andgate1", x = 150, y = 200, w = 100, h = 100)
# Block4 = GateAND_graphical("andgate2", x = 300, y = 200, w = 100, h = 100)
# Block5 = GateAND_graphical("andgate3", x = 300, y = 200, w = 100, h = 100)

blocks = [sysin,  sysout]

# Interaction state
wires = []
drag_start_block = None
drag_start_block = None
drag_start_port_g = None
drag_stop_port_g = None

drag_start_gate_g = None
drag_stop_gate_g = None

truth_lines = None

drag_start_port_g = Vector2(0, 0)
port_drag_pos = Vector2(0, 0)

import pygame
from pygame.math import Vector2

from logic_circuits.pygame_representation import fonts
from logic_circuits.pygame_representation.colors import BG, TEXT, WIRE_HOT
from logic_circuits.pygame_representation.utils import draw_grid, cubic_bezier
from logic_circuits.pygame_representation.wires import Wire

def all_ports():
    return [p for b in blocks for p in b.inputs + b.outputs]

def block_under_mouse(mouse) -> None:
    for b in blocks:
        if b.hover(mouse):
            return b
    return None


def port_under_mouse(mouse, kind=None):
    for p in all_ports():
        if (kind is None or p.kind == kind) and p.hover(mouse):
            return p
    return None


def find_wire_under_mouse(mouse):
    for w in reversed(wires):
        if w.hit_test(mouse):
            return w
    return None


In [6]:
W, H = 900, 600
LIBRARY_WIDTH = 140   # sidebar on the left


In [7]:
class LibraryItem:
    def __init__(self, rect, text, gate_class):
        self.rect = pygame.Rect(rect)
        self.text = text
        self.gate_class = gate_class

    def draw(self, surface):
        pygame.draw.rect(surface, (60, 60, 60), self.rect)
        pygame.draw.rect(surface, (200, 200, 200), self.rect, 2)
        label = fonts.FONT.render(self.text, True, (255, 255, 255))
        surface.blit(label, (self.rect.x+10, self.rect.y+10))

    def hover(self, pos):
        return self.rect.collidepoint(pos)

library_items = [
    LibraryItem((20, 20, 100, 40), "AND", GateAND_graphical),
    LibraryItem((20, 20+50*1, 100, 40), "NOT", GateNOT_graphical),
]

dragging_library_item = None
ghost_gate = None

In [8]:
import io
import contextlib

def truthtable_str(this_gate, sysin):
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        truthtable(this_gate, sysin)
    return buf.getvalue().splitlines()

In [9]:
def cut_wired(wires, gate):
    new_wires = [
        wire for wire in wires if wire.a.gate != gate and wire.b.gate != gate
    ]
    return new_wires

def all_ports():
    return [p for b in blocks for p in b.inputs + b.outputs]

def block_under_mouse(mouse) -> None:
    for b in blocks:
        if b.hover(mouse):
            return b
    return None


def port_under_mouse(mouse, kind=None):
    for p in all_ports():
        if (kind is None or p.kind == kind) and p.hover(mouse):
            return p
    return None


def find_wire_under_mouse(mouse):
    for w in reversed(wires):
        if w.hit_test(mouse):
            return w
    return None

pygame.init()
fonts.init()  # safe to init fonts after pygame.init()

W, H = 900, 600
screen = pygame.display.set_mode((W, H))
pygame.display.set_caption("Wire Editor")
clock = pygame.time.Clock()

from logic_circuits.pygame_representation.button import Button  # if in separate file
play_button = Button(
    rect=(W - 140, 20, 120, 40),   # position + size
    text="Play",
    font=fonts.FONT,
    bg_color=(50, 180, 90),
    fg_color=(255, 255, 255),
    hover_color=(70, 200, 110),
)

# Gate library buttons
gate_buttons = [
    Button(
        rect=(20, 20, 100, 40),
        text="AND",
        font=fonts.FONT,
        bg_color=(100, 100, 220),
        fg_color=(255, 255, 255),
        hover_color=(120, 120, 240),
    ),
    Button(
        rect=(20, 20+50, 100, 40),
        text="NOT",
        font=fonts.FONT,
        bg_color=(220, 100, 100),
        fg_color=(255, 255, 255),
        hover_color=(240, 120, 120),
    ),
    Button(
        rect=(20, 20+2*50, 100, 40),
        text="SysIN",
        font=fonts.FONT,
        bg_color=(100, 100, 220),
        fg_color=(255, 255, 255),
        hover_color=(120, 120, 240),
    ),
    Button(
        rect=(20, 20+3*50, 100, 40),
        text="SysOUT",
        font=fonts.FONT,
        bg_color=(220, 100, 100),
        fg_color=(255, 255, 255),
        hover_color=(240, 120, 120),
    ),
]

running = True
dragging_library_item = None
ghost_gate = None

while running:
    mx, my = pygame.mouse.get_pos()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # -------------------
        # LEFT MOUSE DOWN
        # -------------------
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            # Check if clicked on a library item
            for item in library_items:
                if item.hover((mx, my)):
                    dragging_library_item = item
                    ghost_gate = item.gate_class("ghost", x=mx, y=my, w = 100, h = 100)
                    break
            else:
                # Normal port/gate drag logic
                port_g = port_under_mouse((mx, my))
                gate_g = block_under_mouse((mx, my))

                if port_g and port_g.kind == "out":
                    drag_start_port_g = port_g
                    port_drag_pos = Vector2(mx, my)

                elif gate_g and not port_g:
                    drag_start_gate_g = gate_g
                    block_drag_offset = Vector2(mx - gate_g.rect.x, my - gate_g.rect.y)
        
            for b in blocks:
                if isinstance(b, (SysIN_graphical, GatePass_graphical)):
                    if b.hover_plus((mx, my)):
                        b.increase_ports()
                        wires = cut_wired(wires, b)

                    if b.hover_minus((mx, my)):
                        b.decrease_ports()
                        wires = cut_wired(wires, b)

        # -------------------
        # MOUSE MOVE
        # -------------------
        elif event.type == pygame.MOUSEMOTION:
            if ghost_gate:
                ghost_gate.rect.center = (mx, my)
                ghost_gate._make_ports()
            if drag_start_port_g:
                port_drag_pos = Vector2(mx, my)
            if drag_start_gate_g:
                block_drag_pos = Vector2(mx, my)

        # -------------------
        # LEFT MOUSE UP
        # -------------------
        elif event.type == pygame.MOUSEBUTTONUP and event.button == 1:
            if ghost_gate and dragging_library_item:
                # Only drop if outside library area
                if mx > LIBRARY_WIDTH:
                    new_gate = dragging_library_item.gate_class(
                        f"{dragging_library_item.text.lower()}",
                        x=mx,
                        y=my,
                        w=100, h = 100
                    )
                    blocks.append(new_gate)
                ghost_gate = None
                dragging_library_item = None

            elif drag_start_port_g:
                drag_stop_port_g = port_under_mouse((mx, my), kind="in")
                if drag_stop_port_g and drag_stop_port_g != drag_start_port_g:
                    wires.append(Wire(drag_start_port_g, drag_stop_port_g))
                    connections.append(
                        (
                            drag_start_port_g.gate,
                            drag_stop_port_g.gate,
                            drag_start_port_g.index,
                            drag_stop_port_g.index,
                        )
                    )
                drag_start_port_g = None

            if drag_start_gate_g:
                drag_start_gate_g = None

        # -------------------
        # RIGHT MOUSE DOWN
        # -------------------
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 3:
            if ghost_gate:
                ghost_gate = None
                dragging_library_item = None
            elif drag_start_port_g:
                drag_start_port_g = None
            else:
                # Try wire
                w = find_wire_under_mouse((mx, my))
                if w:
                    wires.remove(w)
                    conn_tuple = (w.a.gate, w.b.gate, w.a.index, w.b.index)
                    if conn_tuple in connections:
                        connections.remove(conn_tuple)
                else:
                    # Try block
                    g = block_under_mouse((mx, my))
                    if g and g in blocks:
                        wires = cut_wired(wires, g)
                        connections[:] = [
                            c for c in connections if c[0] != g and c[1] != g
                        ]
                        if not isinstance(g, (SysIN_graphical, GatePass_graphical)): blocks.remove(g)

        # -------------------
        # ESC to cancel drags
        # -------------------
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
            drag_start_port_g = None
            drag_start_gate_g = None
            ghost_gate = None
            dragging_library_item = None

        # -------------------
        # PLAY BUTTON
        # -------------------
        if play_button.handle_event(event):
            new_gate = make_combined_gate_class(
                "SysOut",
                connections=connections,
                end_gates=[sysout],
                num_in=sysin.num_in,
                num_out=sysout.num_out,
                x=0, y=0
            )
            this_gate = new_gate("or1")
            truth_lines = truthtable_str(this_gate, sysin)  # capture lines

    # -------------------
    # Dragging gates around
    # -------------------
    if drag_start_gate_g:
        drag_start_gate_g.rect.topleft = (
            mx - block_drag_offset.x,
            my - block_drag_offset.y,
        )
        drag_start_gate_g._make_ports()
        if isinstance(drag_start_gate_g, (SysIN_graphical, GatePass_graphical)):
            drag_start_gate_g.plus.topleft = (
                mx - block_drag_offset.x+drag_start_gate_g._w/2,
                my - block_drag_offset.y,
            )
            drag_start_gate_g.minus.topleft = (
                mx - block_drag_offset.x+drag_start_gate_g._w/2,
                my - block_drag_offset.y+drag_start_gate_g._h-drag_start_gate_g._h/10,
            )

    # -------------------
    # Drawing
    # -------------------
    screen.fill(BG)
    draw_grid(screen, W, H)

    # Draw library sidebar
    pygame.draw.rect(screen, (30, 30, 30), (0, 0, LIBRARY_WIDTH, H))
    for item in library_items:
        item.draw(screen)

    # Blocks
    for b_ in blocks:
        b_.draw(screen)

    # Ghost gate being dragged
    if ghost_gate:
        ghost_gate.draw(screen)

    # Wires
    hot_wire = find_wire_under_mouse((mx, my))
    for w in wires:
        w.draw(screen, hot=(w is hot_wire))

    # Live wire being dragged
    if drag_start_port_g:
        p0 = drag_start_port_g.screen_pos()
        p3 = port_drag_pos
        dx = max(40, abs(p3.x - p0.x) * 0.5)
        c1, c2 = Vector2(p0.x + dx, p0.y), Vector2(p3.x - dx, p3.y)
        pts = cubic_bezier(p0, c1, c2, p3, steps=36)
        pygame.draw.lines(screen, WIRE_HOT, False, pts, 3)

        drag_stop_port_g = port_under_mouse((mx, my), kind="in")
        if drag_stop_port_g:
            drag_stop_port_g.draw(screen, hot=True)

        hint = fonts.FONT.render(
            "Release on input to connect (RMB/ESC cancel)", True, TEXT
        )
        screen.blit(hint, (12, 10))

    else:
        hint = fonts.FONT.render(
            "LMB drag block | LMB drag from output→input | RMB delete wire/gate",
            True,
            TEXT,
        )
        screen.blit(hint, (12, 10))

    play_button.draw(screen)
    if truth_lines:
        x, y = LIBRARY_WIDTH + 20, H - 150   # position bottom right
        for i, line in enumerate(truth_lines):
            surf = fonts.FONT.render(line, True, (255, 255, 255), )
            screen.blit(surf, (x, y + i * 20))
    pygame.display.flip()
    clock.tick(60)

pygame.quit()


In [10]:
sysin = SysIN(num_in=2, num_out=2)

notgates = [GateNOT_graphical(f"NOT{idx}", x=0, y=0) for idx in range(3)]
andgates = [GateAND_graphical(f"AND{idx}", x=0, y=0) for idx in range(3)]

OrGate = make_combined_gate_class(
    "OrGate",
    connections=[
        (sysin, andgates[0], 0, 0),
        (sysin, andgates[0], 0, 1),
        (sysin, andgates[1], 1, 0),
        (sysin, andgates[1], 1, 1),
        (andgates[0], notgates[0], 0, 0),
        (notgates[0], andgates[2], 0, 0),
        (andgates[1], notgates[1], 0, 0),
        (notgates[1], andgates[2], 0, 1),
        (andgates[2], notgates[2], 0, 0),
    ],
    end_gates=[notgates[2]],
    num_in=2,
    num_out=1,
    x = 0, y = 0
)

orgate1 = OrGate("or1")

In [11]:
sysin.num_in

2

In [12]:
sysin.set_state([0], [True])
sysin.set_state([0], [True])
orgate1.state

array([ True])

In [13]:
andgates